In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

# Fourier Ptychographic Imaging

##### [1 Forward Imaging Model](fwdimaging.ipynb)

## 2 The Recovery Process

This process recovers a Hi-Res complex image from a Lo-Res image set captured under different incident angles.

> **Note:** A Lo-Res set can be simulated with [fwdimaging.ipynb](fwdimaging.ipynb).

In [ ]:
import numpy as np
import scipy.misc
from PIL import Image
from pathlib import Path, PurePath

In [ ]:
def get_matrix_seq(leds_per_row, xy=False):
    """Returns a reconstruction sequence for a
    leds_per_row-by-leds_per_row matrix. This starts with the center of
    the Fourier spectrum and ends with the image in the edge of the
    spectrum (spiral-shape). If leds_per_row is an even number, the
    sequence starts in (leds_per_row // 2 - 1, leds_per_row // 2 - 1).
    
    Args:
        leds_per_row: The number of LEDs per row (int) of the LED matrix.
        xy: if it is True, the sequence list contains (x, y) tuples. If
            it is False, that contains numbers that enumerate the LEDs
            from the upper left corner (0, 0).
    Returns:
        seq_lst: The sequence list.
    """
    if leds_per_row > 1:
        seq_lst = list()
        arr = np.array(range(1, leds_per_row ** 2 + 1))
        # Split arr (numpy.ndarray) to get a n-by-n array.
        arr = np.split(arr, leds_per_row)
        arr = np.array(arr)  # numpy.split returns a list of numpy.ndarrays.
        if leds_per_row % 2 == 0:
            x = leds_per_row // 2 - 1
        else:
            x = leds_per_row // 2
        y = x
        if xy:
            seq_lst.append((x, y))
        else:
            seq_lst.append(arr[y][x])
        c = 1
        while True:
            for i in range(2):
                toward = (1, -1)[i % 2]
                for j in range(2):
                    # ('x', 'y')[j % 2]
                    for k in range(c):
                        if ('x', 'y')[j % 2] == 'x':
                            x += toward
                        else:
                            y += toward
                        if xy:
                            seq_lst.append((x, y))
                        else:
                            seq_lst.append(arr[y][x])
                        if (((leds_per_row % 2 == 0) and
                             (x, y) == (0, leds_per_row - 1)) or
                            ((leds_per_row % 2 != 0) and
                             (x, y) == (leds_per_row - 1, 0))):
                            return seq_lst
                c += 1
        return None

In [ ]:
def gen_hires_img(lores_set, sequence, hires_w, hires_h, cft, lores_w,
                  lores_h, dkx, dky, kx, ky, output_path, loops=5,
                 **kwargs):
    """Generates a Hi-Res image from a Lo-Res image set by applying
    of the recovery process of Fourier Ptychography.
    
    Args:
        lores_set: The Lo-Res image set as a 3-dimentional array
            (numpy.ndarray).
        sequence: The reconstruction sequence (list of numbers).
        hires_w: The width of the Hi-Res output image in pixels (int).
        hires_h: The height of the Hi-Res output image in pixels (int).
        cft: The coherent transfer function (numpy.ndarray).
        lores_w: The width of the Lo-Res output images in pixels (int).
        lores_h: The height of the Lo-Res output images in pixels (int).
        dkx: (float).
        dky: (float).
        kx: A numpy.ndarray with the x-components of k_0 * wave vectors.
        ky: A numpy.ndarray with the y-components of k_0 * wave vectors.
        output_path: The directory path of the reconstruction.
        loops: Number of loops in the reconstruction process.
        **kwargs: Keyword arguments.
    """
    # Name for the output image without the file extension.
    name = kwargs.pop('name', 'fp-reconstruction')
    img_format = kwargs.pop('format', 'TIF')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_lores_set.__name__,
                  list(kwargs.keys())[-1]))
    dir_path = Path(output_path)
    Path.mkdir(dir_path, exist_ok=True, parents=True)
    obj_recover = np.ones((hires_h, hires_w), dtype=int)
    obj_recover_ft = np.fft.fftshift(np.fft.fft2(obj_recover))
    for i in range(loops):
        for j in sequence:
            kxc = round_half_up((hires_w + 1) / 2 + kx[j - 1] / dkx)
            kyc = round_half_up((hires_h + 1) / 2 + ky[j - 1] / dky)
            kxl = round_half_up(kxc - (lores_w - 1) / 2)
            kyl = round_half_up(kyc - (lores_h - 1) / 2)
            kxh = round_half_up(kxc + (lores_w - 1) / 2)
            kyh = round_half_up(kyc + (lores_h - 1) / 2)
            lores_ft = (((lores_w / hires_w) ** 2) * 
                         obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] *
                        cft)
            lores_img = np.fft.ifft2(np.fft.ifftshift(lores_ft))
            lores_img = (((hires_w / lores_w) ** 2) *
                         lores_set[j - 1] *
                         np.exp(1j * np.angle(lores_img)))
            lores_ft = np.fft.fftshift(np.fft.fft2(lores_img)) * cft
            obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] = ((1 - cft) *
                obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] + lores_ft)
    # Transform back the converged solution in the Fourier space
    # to the spatial domain.
    obj_recover = np.fft.ifft2(np.fft.ifftshift(obj_recover_ft))
    obj_recover = np.absolute(obj_recover)
    filename = name.split('.')[0] +  '.' + img_format.lower()
    file_path = PurePath(dir_path, filename).as_posix()
    # Save and prevent the rescaling of the dynamic range.
    scipy.misc.toimage(obj_recover, cmin=0, cmax=255).save(file_path)